In [4]:
!pip install pylidc
!pip install pydicom
!pip install opencv-python

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
import numpy as np
import pylidc as pl
import pydicom
import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score


In [15]:
import os
import pydicom
import pylidc as pl
import numpy as np
import cv2
from pylidc.Contour import Contour
from statistics import median_high

# Paths to the directories containing DICOM files
base_dirs = [
    "/kaggle/input/lidcidri30/LIDC-IDRI-0001-0200",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0201-0400",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0401-0600"
]

images = []
labels = []
malignant_count = 0
benign_count = 0
max_malignant_count = 4500  # Target number of malignant images
max_benign_count = 2500     # Target number of benign images

# Patch the Contour class method to_matrix
def patched_to_matrix(self, include_k=True):
    coords = np.array([list(map(int, coord.split(','))) for coord in self.coords.split('\n')])
    ij = np.c_[coords[:, 0], coords[:, 1]]

    if not include_k:
        return ij.astype(np.int64)

    k = np.ones(ij.shape[0]) * self.image_z_position
    return np.c_[ij, k].astype(np.int64)

Contour.to_matrix = patched_to_matrix

def calculate_malignancy(nodule):
    list_of_malignancy = [annotation.malignancy for annotation in nodule]
    malignancy = median_high(list_of_malignancy)
    return 'malignant' if malignancy > 3 else 'benign'

# Process DICOM files with the patched Contour class
for base_dir in base_dirs:
    for dirpath, _, filenames in os.walk(base_dir):
        for filename in filenames:
            if filename.endswith(".dcm"):
                try:
                    dicom_file = os.path.join(dirpath, filename)
                    dicom_data = pydicom.dcmread(dicom_file)
                    pid = dicom_data.PatientID
                    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                    if scan is None:
                        continue

                    nodules_annotation = scan.cluster_annotations()

                    try:
                        if len(nodules_annotation) > 0:
                            contains_malignant = False

                            for nodule in nodules_annotation:
                                malignancy = calculate_malignancy(nodule)
                                if malignancy == 'malignant':
                                    contains_malignant = True
                                    break

                            if contains_malignant:
                                if malignant_count >= max_malignant_count:
                                    continue
                                malignancy = 'malignant'
                                malignant_count += 1
                            else:
                                if benign_count >= max_benign_count:
                                    continue
                                malignancy = 'benign'
                                benign_count += 1

                            img = dicom_data.pixel_array  # Assuming DICOM contains pixel data directly
                            img = cv2.resize(img, (224, 224))  
                            img = img.astype(np.float32)

                            images.append(img)
                            labels.append(malignancy)

                            print(f"Processed {malignant_count + benign_count} files")
                            print(f"Malignancy: {malignancy}")
                            print(f"Patient ID: {pid}")

                        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
                            break
                    except Exception as e:
                        # Skip problematic nodules that cannot be grouped manually
                        print(f"Skipping nodule processing for Patient ID {pid} due to error: {e}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
            break

print(f"Total malignant images: {malignant_count}")
print(f"Total benign images: {benign_count}")


Processed 1 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 2 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 3 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 4 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 5 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 6 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 7 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 8 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 9 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 10 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 11 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 12 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 13 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 14 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 15 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 16 files
Malignancy: ben

In [16]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [17]:
# Normalization
images = images/255.0

In [18]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'benign': 0, 'malignant': 1}


In [19]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 224, 224, 1)
X_test = np.array(X_test).reshape(-1, 224, 224, 1)

# Define number of classes
num_classes = len(np.unique(labels_encoded))

print(num_classes)

2


In [20]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Add, Flatten, Dense,ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform


def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Saving the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First layer
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second layer
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def ResNet50(input_shape=(64, 64, 1), classes=2):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

model = ResNet50(input_shape=(224, 224, 1), classes=2)


In [21]:
model.summary()

Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 112, 112,  │      3,200 │ zero_padding2d_3… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_conv1            │ (None, 112, 112,  │        256 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_147      │ (None, 112, 112,  │          0 │ bn_conv1[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 55, 55,    │          0 │ activation_147[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2a      │ (None, 55, 55,    │      4,160 │ max_pooling2d_3[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2a       │ (None, 55, 55,    │        256 │ res2a_branch2a[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_148      │ (None, 55, 55,    │          0 │ bn2a_branch2a[0]… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2b      │ (None, 55, 55,    │     36,928 │ activation_148[0… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2b       │ (None, 55, 55,    │        256 │ res2a_branch2b[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_149      │ (None, 55, 55,    │          0 │ bn2a_branch2b[0]… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2c      │ (None, 55, 55,    │     16,640 │ activation_149[0… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch1       │ (None, 55, 55,    │     16,640 │ max_pooling2d_3[… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2c       │ (None, 55, 55,    │      1,024 │ res2a_branch2c[0… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch1        │ (None, 55, 55,    │      1,024 │ res2a_branch1[0]… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_48 (Add)        │ (None, 55, 55,    │          0 │ bn2a_branch2c[0]

 Total params: 23,618,306 (90.10 MB)

 Trainable params: 23,565,186 (89.89 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [25]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model

# Assuming ResNet50 and other utility functions (identity_block, convolutional_block) are defined as in your previous code.

# Number of folds
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Placeholder array to store performance metrics
accuracy_scores = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Fold {fold + 1}/{k}')
    
    # Split data into training and validation sets for the fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Initialize a new model for each fold
    model = ResNet50(input_shape=(224, 224, 1), classes=2)
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Filepath to save the best model of this fold
    checkpoint_filepath = f'best_resnet50_model_fold_{fold + 1}.keras'
    
    # ModelCheckpoint to save the best weights
    checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                 monitor='val_accuracy', 
                                 save_best_only=True, 
                                 mode='max', 
                                 verbose=1)

    # EarlyStopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=4,  # Stop training if no improvement after 4 epochs
                                   mode='max',
                                   verbose=1,
                                   restore_best_weights=True)
    
    # Train the model with early stopping and checkpoint callbacks
    model.fit(X_train_fold, y_train_fold, 
              epochs=5,  # You can increase the number of epochs if needed
              batch_size=16, 
              validation_data=(X_val_fold, y_val_fold),
              verbose=1,
              callbacks=[checkpoint, early_stopping])

    # Load the best model saved during this fold
    model = load_model(checkpoint_filepath)

    # Evaluate the model on the validation set
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracy_scores.append(scores[1])  # Save validation accuracy from the evaluation

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across {k}-folds: {avg_accuracy:.4f}')


Fold 1/5
Epoch 1/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.6025 - loss: 2.2827
Epoch 1: val_accuracy improved from -inf to 0.72857, saving model to best_resnet50_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 108s 192ms/step - accuracy: 0.6028 - loss: 2.2785 - val_accuracy: 0.7286 - val_loss: 1.8163
Epoch 2/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.8389 - loss: 0.3697
Epoch 2: val_accuracy improved from 0.72857 to 0.86964, saving model to best_resnet50_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 48s 173ms/step - accuracy: 0.8392 - loss: 0.3693 - val_accuracy: 0.8696 - val_loss: 6.3734
Epoch 3/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.9597 - loss: 0.1252
Epoch 3: val_accuracy improved from 0.86964 to 0.99107, saving model to best_resnet50_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 48s 173ms/step - accuracy: 0.9597 - loss: 0.1251 - val_accuracy: 0.9911 - val_loss: 0.2790
Epoch 4/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - ac